In [4]:
import torch
from torch.utils.data import Dataset
import os
import cv2
import numpy as np
import torch
from data_preprocessing import apply_color_space_conversion, apply_time_domain_normalization, add_white_noise

NameError: name 'audio_classifier' is not defined

In [2]:


class UBFCrPPGDataset(Dataset):
    def __init__(self, data_dir, split='train', transform=None, fps=30, max_frames=1387):
        """
        Args:
            data_dir (str): Путь к директории с данными UBFC-rPPG.
            split (str): 'train' или 'test' для выбора подмножества (30 субъектов для train, 12 для test).
            transform (callable, optional): Дополнительные трансформации для данных.
            fps (int): Частота кадров видео (по умолчанию 30 fps).
            max_frames (int): Максимальное количество кадров для обработки (по умолчанию 1387).
        """
        self.data_dir = data_dir
        self.split = split
        self.transform = transform
        self.fps = fps
        self.max_frames = max_frames
        self.subjects = sorted(os.listdir(data_dir))  # Список директорий субъектов
        self.num_subjects = len(self.subjects)
        
        # Разделение на train/test (30 для train, 12 для test)
        if split == 'train':
            self.subjects = self.subjects[:12]
        elif split == 'test':
            self.subjects = self.subjects[12:15]
        else:
            raise ValueError("split должен быть 'train' или 'test'")
        
        self.video_paths = []
        self.gt_paths = []
        
        # Собираем пути к видео и ground truth файлам
        for subject in self.subjects:
            subject_dir = os.path.join(data_dir, subject)
            if os.path.isdir(subject_dir):
                video_path = os.path.join(subject_dir, 'vid.avi')  # Предполагаемый путь к видео
                gt_path = os.path.join(subject_dir, 'ground_truth.txt')
                if os.path.exists(video_path) and os.path.exists(gt_path):
                    self.video_paths.append(video_path)
                    self.gt_paths.append(gt_path)
        
        self.num_samples = len(self.video_paths)
        if self.num_samples == 0:
            raise ValueError("Не найдено видео или ground truth файлов в указанной директории")

    def __len__(self):
        """Возвращает количество образцов в датасете."""
        return self.num_samples

    def __getitem__(self, idx):
        """
        Возвращает один образец данных: пространственно-временную карту и ground truth данные.
        
        Args:
            idx (int): Индекс образца.
        
        Returns:
            dict: Словарь с полями:
                - 'spatial_temporal_map': torch.Tensor размером (C=3, R=4, T) - нормализованная карта.
                - 'ppg_signal': torch.Tensor - PPG сигнал из ground truth.
                - 'hr_values': torch.Tensor - Значения HR из ground truth.
                - 'timesteps': torch.Tensor - Временные метки.
        """
        # Загружаем видео
        video_path = self.video_paths[idx]
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            raise ValueError(f"Не удалось открыть видео: {video_path}")
        
        frames = []
        frame_count = 0
        while frame_count < self.max_frames:
            ret, frame = cap.read()
            if not ret:
                break
            frames.append(frame)
            frame_count += 1
        cap.release()
        
        # Проверка количества кадров
        if len(frames) == 0:
            raise ValueError(f"Видео пустое: {video_path}")
        if len(frames) < self.max_frames:
            # Повторяем последний кадр, если видео короче
            frames.extend([frames[-1]] * (self.max_frames - len(frames)))
        elif len(frames) > self.max_frames:
            frames = frames[:self.max_frames]
        
        # Предобработка: создаём пространственно-временную карту
        spatial_temporal_map = []
        for frame in frames:
            # Применяем конверсию в YUV для каждого кадра
            facial_means_yuv = apply_color_space_conversion(frame)
            if facial_means_yuv is None:
                # Если регион не определён, заполняем нули
                facial_means_yuv = {
                    'between_eyebrows_landmarks': [0, 0, 0],
                    'nose_landmarks': [0, 0, 0],
                    'left_cheek_landmarks': [0, 0, 0],
                    'right_cheek_landmarks': [0, 0, 0]
                }
            spatial_temporal_map.append(facial_means_yuv)
        
        # Нормализация во временной области
        normalized_map = apply_time_domain_normalization(spatial_temporal_map)
        
        # Добавление белого шума
        noisy_map = add_white_noise(normalized_map, noise_std=0.05)
        
        # Преобразуем в torch.Tensor
        spatial_temporal_map_tensor = torch.from_numpy(noisy_map).float()
        
        # Загружаем ground truth данные
        gt_path = self.gt_paths[idx]
        with open(gt_path, 'r') as f:
            data = [float(x) for x in f.read().split()]
        
        # Предполагаем, что данные разделены на три равные части
        T = self.max_frames  # Должно совпадать с количеством кадров
        ppg_signal = data[:T]
        hr_values = data[T:2*T]
        timesteps = data[2*T:3*T]
        
        # Проверка длины ground truth
        if len(ppg_signal) < T:
            ppg_signal.extend([ppg_signal[-1]] * (T - len(ppg_signal)))
        elif len(ppg_signal) > T:
            ppg_signal = ppg_signal[:T]
        
        if len(hr_values) < T:
            hr_values.extend([hr_values[-1]] * (T - len(hr_values)))
        elif len(hr_values) > T:
            hr_values = hr_values[:T]
        
        if len(timesteps) < T:
            timesteps.extend([timesteps[-1]] * (T - len(timesteps)))
        elif len(timesteps) > T:
            timesteps = timesteps[:T]
        
        # Преобразуем ground truth в torch.Tensor
        ppg_tensor = torch.tensor(ppg_signal, dtype=torch.float32)
        hr_tensor = torch.tensor(hr_values, dtype=torch.float32)
        timesteps_tensor = torch.tensor(timesteps, dtype=torch.float32)
        
        # Применяем дополнительные трансформации, если указаны
        if self.transform is not None:
            sample = {
                'spatial_temporal_map': spatial_temporal_map_tensor,
                'ppg_signal': ppg_tensor,
                'hr_values': hr_tensor,
                'timesteps': timesteps_tensor
            }
            sample = self.transform(sample)
            return sample
        
        return {
            'spatial_temporal_map': spatial_temporal_map_tensor,
            'ppg_signal': ppg_tensor,
            'hr_values': hr_tensor,
            'timesteps': timesteps_tensor
        }

    def get_subject_info(self):
        """Возвращает информацию о субъектах в датасете."""
        return {
            'split': self.split,
            'num_subjects': len(self.subjects),
            'subjects': self.subjects,
            'num_samples': self.num_samples
        }

KeyboardInterrupt: 